In [1]:
import numpy as np


class RankingSimilarity(object):
    """
    This class will include some similarity measures between two different
    ranked lists.
    """

    def __init__(self, S, T, verbose=False):
        """
        Initialize the object with the required lists.
        Examples of lists:
        S = ['a', 'b', 'c', 'd', 'e']
        T = ['b', 'a', 1, 'd']
        Both lists relfect the ranking of the items of interest, for example,
        list S tells us that item 'a' is ranked first, 'b' is ranked second, etc.
        Args:
            S, T (list or numpy array): lists with alphanumeric elements. They
                could be of different lengths. Both of the them should be
                ranked, i.e., each element's position reflects its respective
                ranking in the list. Also we will require that there is no
                duplicate element in each list.
        """

        assert(type(S) in [list, np.ndarray])
        assert(type(T) in [list, np.ndarray])

        assert(len(S) == len(set(S)))
        assert(len(T) == len(set(T)))

        self.S, self.T = S, T
        self.N_S, self.N_T = len(S), len(T)
        self.verbose = verbose
        self.p = 0.5  # just a place holder

    def _bound_range(self, value):
        """Bounds the value to [0.0, 1.0].
        """

        try:
            assert(0 <= value <= 1 or np.isclose(1, value))
            return value

        except AssertionError:
            print('Value out of [0, 1] bound, will bound it.')
            larger_than_zero = max(0.0, value)
            less_than_one = min(1.0, larger_than_zero)
            return less_than_one

    def rbo(self, k=None, p=1.0, ext=False):
        """
        This the weighted non-conjoint measures, namely, rank-biased overlap.
        Unlike Kendall tau which is correlation based, this is intersection
        based.
        The implementation if from Eq. (4) or Eq. (7) (for p != 1) from the RBO
        paper: http://www.williamwebber.com/research/papers/wmz10_tois.pdf
        If p=1, it returns to the un-bounded set-intersection overlap, according
        to Fagin et al.
        https://researcher.watson.ibm.com/researcher/files/us-fagin/topk.pdf
        The fig. 5 in that RBO paper can be used as test case.
        Note there the choice of p is of great importance, since it essentically
        control the 'top-weightness'. Simply put, to an extreme, a small p value
        will only consider first few items, whereas a larger p value will
        consider more itmes. See Eq. (21) for quantitative measure.
        Args:
            k (int), default None: The depth of evaluation.
            p (float), default 1.0: Weight of each agreement at depth d:
                p**(d-1). When set to 1.0, there is no weight, the rbo returns
                to average overlap.
            ext (Boolean) default False: If True, we will extropulate the rbo,
                as in Eq. (23)
        Returns:
            The rbo at depth k (or extrapolated beyond)
        """

        if not self.N_S and not self.N_T:
            return 1  # both lists are empty

        if not self.N_S or not self.N_T:
            return 0  # one list empty, one non-empty

        if k is None:
            k = float('inf')
        k = min(self.N_S, self.N_T, k)

        # initilize the agreement and average overlap arrays
        A, AO = [0] * k, [0] * k
        if p == 1.0:
            weights = [1.0 for _ in range(k)]
        else:
            assert(0.0 < p < 1.0)
            weights = [1.0 * (1 - p) * p**d for d in range(k)]

        self.p = p

        # using dict for O(1) look up
        S_running, T_running = {self.S[0]: True}, {self.T[0]: True}
        A[0] = 1 if self.S[0] == self.T[0] else 0
        AO[0] = weights[0] if self.S[0] == self.T[0] else 0

        PP = ProgressPrintOut(k) if self.verbose else NoPrintOut()
        for d in range(1, k):

            PP.printout(d, delta=1)
            tmp = 0
            # if the new item from S is in T already
            if self.S[d] in T_running:
                tmp += 1
            # if the new item from T is in S already
            if self.T[d] in S_running:
                tmp += 1
            # if the new items are the same, which also means the previous
            # two cases didn't happen
            if self.S[d] == self.T[d]:
                tmp += 1

            # update the agreement array
            A[d] = 1.0 * ((A[d - 1] * d) + tmp) / (d + 1)

            # update the average overlap array
            if p == 1.0:
                AO[d] = ((AO[d - 1] * d) + A[d]) / (d + 1)
            else:  # weighted average
                AO[d] = AO[d - 1] + weights[d] * A[d]

            # add the new item to the running set (dict)
            S_running[self.S[d]] = True
            T_running[self.T[d]] = True

        if ext and p < 1:
            return self._bound_range(AO[-1] + A[-1] * p**k)
        else:
            return self._bound_range(AO[-1])

    def rbo_ext(self, p=0.98):
        """
        This is the ultimate implementation of the rbo, namely, the extrapolated
        version. The corresponding formula is Eq. (32) in the rbo paper
        """

        assert(0.0 < p < 1.0)
        self.p = p

        if not self.N_S and not self.N_T:
            return 1  # both lists are empty

        if not self.N_S or not self.N_T:
            return 0  # one list empty, one non-empty

        # since we are dealing with un-even lists, we need to figure out the
        # long (L) and short (S) list first. The name S might be confusing
        # but in this function, S refers to short list, L refers to long list
        if len(self.S) > len(self.T):
            L, S = self.S, self.T
        else:
            S, L = self.S, self.T

        s, l = len(S), len(L)

        # initilize the overlap and rbo arrays
        # the agreement can be simply calculated from the overlap
        X, A, rbo = [0] * l, [0] * l, [0] * l

        # first item
        S_running, L_running = {S[0]}, {L[0]}  # for O(1) look up
        X[0] = 1 if S[0] == L[0] else 0
        A[0] = X[0]
        rbo[0] = 1.0 * (1 - p) * A[0]

        # start the calculation
        PP = ProgressPrintOut(l) if self.verbose else NoPrintOut()
        disjoint = 0
        ext_term = A[0] * p

        for d in range(1, l):
            PP.printout(d, delta=1)

            if d < s:  # still overlapping in length

                S_running.add(S[d])
                L_running.add(L[d])

                # again I will revoke the DP-like step
                overlap_incr = 0  # overlap increament at step d

                # if the new itmes are the same
                if S[d] == L[d]:
                    overlap_incr += 1
                else:
                    # if the new item from S is in L already
                    if S[d] in L_running:
                        overlap_incr += 1
                    # if the new item from L is in S already
                    if L[d] in S_running:
                        overlap_incr += 1

                X[d] = X[d - 1] + overlap_incr
                # Eq. (28) that handles the tie. len() is O(1)
                A[d] = 2.0 * X[d] / (len(S_running) + len(L_running))
                rbo[d] = rbo[d - 1] + 1.0 * (1 - p) * (p**d) * A[d]

                ext_term = 1.0 * A[d] * p**(d + 1)  # the extropulate term

            else:  # the short list has fallen off the cliff
                L_running.add(L[d])  # we still have the long list

                # now there is one case
                overlap_incr = 1 if L[d] in S_running else 0

                X[d] = X[d - 1] + overlap_incr
                A[d] = 1.0 * X[d] / (d + 1)
                rbo[d] = rbo[d - 1] + 1.0 * (1 - p) * (p**d) * A[d]

                X_s = X[s - 1]  # this the last common overlap
                # second term in first parenthesis of Eq. (32)
                disjoint += 1.0 * (1 - p) * (p**d) * \
                    (X_s * (d + 1 - s) / (d + 1) / s)
                ext_term = 1.0 * ((X[d] - X_s) / (d + 1) + X[s - 1] / s) * \
                    p**(d + 1)  # last term in Eq. (32)

        return self._bound_range(rbo[-1] + disjoint + ext_term)

    def top_weightness(self, p=None, d=None):
        """
        This function will evaluate the degree of the top-weightness of the rbo.
        It is the implementation of Eq. (21) of the rbo paper.
        As a sanity check (per the rbo paper),
        top_weightness(p=0.9, d=10) should be 86%
        top_weightness(p=0.98, d=50) should be 86% too
        Args:
            p (float), defalut None: A value between zero and one.
            d (int), default None: Evaluation depth of the list.
        Returns:
            A float between [0, 1], that indicates the top-weightness.
        """

        # sanity check
        if p is None:
            p = self.p
        assert (0. < p < 1.0)

        if d is None:
            d = min(self.N_S, self.N_T)
        else:
            d = min(self.N_S, self.N_T, int(d))

        if d == 0:
            top_w = 1
        elif d == 1:
            top_w = 1 - 1 + 1.0 * (1 - p) / p * (np.log(1.0 / (1 - p)))
        else:
            sum_1 = 0
            for i in range(1, d):
                sum_1 += 1.0 * p**(i) / i
            top_w = 1 - p**(i) + 1.0 * (1 - p) / p * (i + 1) * \
                (np.log(1.0 / (1 - p)) - sum_1)  # here i == d-1

        if self.verbose:
            print('The first {} ranks have {:6.3%} of the weight of '
                  'the evaluation.'.format(d, top_w))

        return self._bound_range(top_w)


class ProgressPrintOut(object):
    """Quick status print out.
    """

    def __init__(self, N):
        self._old = 0
        self._total = N

    def printout(self, i, delta=10):
        # print out progess every delta %
        cur = 100 * i // self._total
        if cur >= self._old + delta:
            print('\r', 'Current progress: {} %...'.format(cur), end='')
            self._old = cur
        if i == self._total - 1:
            print('\r', 'Current progress: 100 %...', end='')
            print('\nFinished!')

        return


class NoPrintOut(object):
    def printout(self, i, delta=10):
        pass

In [1]:
# S = ['ab', 'b', 'd']
# T = ['a', 'b', 'dc']
# RankingSimilarity(S, T).rbo_ext(p=0.9)

In [2]:
import shapSD as ssd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Display all content in dataframe
pd.set_option('display.max_colwidth', -1)
np.random.seed(42)

Using TensorFlow backend.


In [6]:
file_path = '../data/adult.csv'
original_adult = pd.read_csv(file_path, index_col=0)
df_adult = ssd.DataEncoder(original_adult.drop('income', axis=1)).label_encoding()
df_adult = ssd.FeatureProcessing(df_adult).data_scaling()
x, y = df_adult.drop(['education'],axis = 1 ), original_adult['income']
x_adult_train, x_adult_test, y_adult_train, y_adult_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test_origin = original_adult.drop('income', axis=1).iloc[x_adult_test.index]
y_adult_train = pd.get_dummies(y_adult_train, drop_first=True)
y_adult_test = pd.get_dummies(y_adult_test, drop_first=True)

nn_model = ssd.InitializeModel(x_adult_train, y_adult_train).keras_nn_model()

In [7]:
# def subgroup_discovery(df_effect, target_name, label_name, selected_attr, measure, inverse_effect=False,
#                            statistic_is_positive=True):
#     target = ssd.NumericTarget(target_name)
#     search_space = ssd.create_selectors(df_effect, nbins=10, ignore=[target_name, label_name, selected_attr])
#     task = ssd.SubgroupDiscoveryTask(df_effect, target, search_space, qf=measure, result_set_size=20, depth=3)
#     result = ssd.BeamSearch().execute(task, inverse_effect, statistic_is_positive)
#     df_result = ssd.as_df(df_effect, result, statistics_to_show=ssd.all_statistics_numeric)
#     return df_result[['quality', 'subgroup', 'size_sg', 'mean_sg', 'mean_dataset', 'mean_lift']]

In [8]:
def calc_rank_biased_overlap(attr):
    local_exp = ssd.LocalExplainer(x_adult_test, nn_model)
    pattern_exp = ssd.PatternExplainer(x_test_origin, x_adult_test, attribute=attr, model=nn_model, 
                              local_exp=local_exp)
    df_lime = pd.read_csv('../data/adult_lime_weights_nn_model.csv', sep='\t')
    df_lime_effect = x_test_origin.copy()
    df_lime_effect.reset_index(drop=True, inplace=True)
    lime_target_name = '{}_lime_weight'.format(attr)
    df_lime_effect[lime_target_name] = df_lime[attr]
    pattern_exp.effect_name = lime_target_name
    df_lime_result = pattern_exp.subgroup_discovery(df_lime_effect, result_set_size=20)
    lime_subgroups = list(df_lime_result['subgroup'].values)
    
    df_shap = pd.read_csv('../data/adult_shap_value_nn_model.csv', sep='\t')
    df_shap_effect = x_test_origin.copy()
    df_shap_effect.reset_index(drop=True, inplace=True)
    shap_target_name = '{}_shap_values'.format(attr)
    df_shap_effect[shap_target_name] = df_shap[attr]
    pattern_exp.effect_name = shap_target_name
    df_shap_result = pattern_exp.subgroup_discovery(df_shap_effect, result_set_size=20)
    shap_subgroups = list(df_shap_result['subgroup'].values)
    
    r_similarity = RankingSimilarity(lime_subgroups, shap_subgroups)
    # p=0.8, top5 are given 86% weight; p=0.9, top10 are given 85% weight; p=0.95, top30 are given 85% weight
    scores = [r_similarity.rbo_ext(p=0.8), r_similarity.rbo_ext(p=0.9), r_similarity.rbo_ext(p=0.95)]
    print('attribute: ', attr, 'scores: ', scores)
    return scores

In [9]:
rbo_scores = {}
for attr in x_adult_test.columns:
    rbo_scores[attr] = calc_rank_biased_overlap(attr=attr)

attribute:  age scores:  [0.8529479261463331, 0.8180181917197191, 0.7921633747300507]
attribute:  work-class scores:  [0.0, 0.0, 0.0]
attribute:  fnlwgt scores:  [0.0, 0.0, 0.0]
attribute:  education-num scores:  [0.9245638628841623, 0.8557346789181646, 0.7928860785497591]
attribute:  marital-status scores:  [0.7664761043804399, 0.6628783398521207, 0.6188392782579595]
attribute:  occupation scores:  [0.12246292753272357, 0.11256213646808358, 0.09124976115876395]
attribute:  relationship scores:  [0.32492585506544713, 0.27012427293616714, 0.20624952231752788]
attribute:  race scores:  [0.0, 0.0, 0.0]
attribute:  sex scores:  [0.9462200758002189, 0.9073873426027799, 0.8680043601804008]
attribute:  capital-gain scores:  [0.0, 0.0, 0.0]
attribute:  capital-loss scores:  [0.0, 0.0, 0.0]
attribute:  hours-per-week scores:  [0.05535666203320279, 0.07580787691559551, 0.08656533333251937]
attribute:  native-country scores:  [0.0, 0.0, 0.0]


In [10]:
df_rbo = pd.DataFrame(rbo_scores).transpose()
df_rbo.columns = ['p=0.8', 'p=0.9', 'p=0.95']
df_rbo

,p=0.8,p=0.9,p=0.95
age,0.852948,0.818018,0.792163
work-class,0.000000,0.000000,0.000000
fnlwgt,0.000000,0.000000,0.000000
education-num,0.924564,0.855735,0.792886
marital-status,0.766476,0.662878,0.618839
occupation,0.122463,0.112562,0.091250
relationship,0.324926,0.270124,0.206250
race,0.000000,0.000000,0.000000
sex,0.946220,0.907387,0.868004
capital-gain,0.000000,0.000000,0.000000
